# Name Generator

This generator generates pronounceable names up to 4 syllables. Letters have equal probablity of being chosen. However, if two consonants are selected to appear one after the other, how synergetic they are together is taken into account.

The names follow the pattern consonant (c) -> vowel (v) -> c -> v -> c (or vice versa). Each stage adds either 1 or 2 letters--this is also randomly chosen.

All random numbers are generated through quantum circuit measurements implemented with Qiskit.


In [1]:
from qiskit import *
import math

In [17]:
#required dictionaries
cons = ['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','w','x','y','z'] #consonants
vows = ['a','u','e','i','o'] #vowels
cons_or_vow = {'c': cons, 'v': vows} # consonants or vowels
letter_type_dict = {'0':'c', '1':'v'}
reverse_letter_type_dict = {'c':'0', 'v':'1'}

letter_synergy_dict = {'b': ['l','r','w','y'], 
                       'c':['h', 'k', 'l', 'r','w','y'],
                       'd':['h','r','w','y'],
                       'f':['f','h','l','r','y'],
                       'g':['h','y'],
                       'h': ['m','y'], 
                       'j':['h','j','r','w','y'],
                       'k':['h','y'],
                       'l':['l','y'],
                       'm':['h','y'], 
                       'n':['n','w','y'],
                       'p':['p','h','l','r','w','y'],
                       'q':['k','w','y'],
                       'r':['h','r','y'],
                       's':['h','l','p','r','s','t','v','w','y'],
                       't':['h','r', 't', 'v','w','y','z'],
                       'v':['h','l','v','y'],
                       'w':['h','w'],
                       'x':['l','r','w','x','y'],
                       'y':['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','x','z'],
                       'z':['h','l','r','y','z']
                       } # which consonants are synergetic when together? Not symmetric. Key letter followed by any of the letters in its value list.



In [18]:
# the quantum part

def run_QuantumCirc(n): # creates and runs a quantum circuit with a Hadamard operating on each qubit
    qr = qiskit.QuantumRegister(n) # create quantum register with n qubits
    cr = qiskit.ClassicalRegister(n) # create classical register with n bits
    circ  = qiskit.QuantumCircuit(qr, cr) # create circuit with the two registers
    circ.h(qr) # perform Hadamard  on each qubit
    circ.measure(qr,cr) # each qubit is measured, and the outcome for one qubit is either 0 or 1
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') ) 
    result = job.result().get_counts() 
    return result

def random_quantum(num_choices): #returns random int between 0 and num_choices (not inclusive of num_choices)
    if math.log(num_choices,2)%int(math.log(num_choices,2)) == 0: # checks whether log(num_choices) to the base 2 is a whole number
        n = int(math.log(num_choices,2)) # n = number of qubits
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get) 
    else: 
        n = int(math.log(num_choices,2))+1 # adds 1 to log(N) to the base 2 to get total number of qubits needed
        result = run_QuantumCirc(n)
        max_res = max(result, key=result.get)
        while(int(max_res,2) > num_choices-1): # find max that is less than num_choices
            result.pop(str(max_res))
            max_res = max(result, key=result.get)
    random_number = int(max_res,2)
    return random_number

In [19]:
# generating the letters and eventually the name

def return_letters(c_or_v, num_letters): # returns one or two letters
    letter_array = cons_or_vow[c_or_v] #are we dealing with consonants or vowels?
    if num_letters == 1: #r eturning one letter
        ind = random_quantum(len(letter_array))
        return letter_array[ind]
    else: #returning two letters
        ind1 = random_quantum(len(letter_array))
        ind2 = random_quantum(len(letter_array))
        if c_or_v == 'c': #if we're dealing with consonants, need to check whether the 2 are synergetic
            if letter_array[ind2] in letter_synergy_dict[letter_array[ind1]]:
                return letter_array[ind1]+letter_array[ind2]
            else:
                return letter_array[ind1]
        else:
            return letter_array[ind1]+letter_array[ind2]

def make_name(): #names are made in the pattern c>v>c>v>c etc. each can be one or two letters
    syllables = random_quantum(4) + 1 # names with up to 4 syllables
    word = []
    letter_type = letter_type_dict[str(random_quantum(2))] #selects whether the opening letters are consonants or vowels
    word += return_letters(letter_type,random_quantum(2)+1) #generates first part
    while syllables >0: #generates the rest of the syllables
        letter_type = letter_type_dict[str(int(reverse_letter_type_dict[letter_type])^1)] # toggles c -> v and v->c, ^ is xor
        word += return_letters(letter_type,random_quantum(2)+1)
        syllables-=1
    
    return str(''.join(word)).capitalize()

In [20]:
print(make_name())
print(make_name())
print(make_name())
print(make_name())
print(make_name())
print(make_name())
print(make_name())
print(make_name())

Opaac
Eca
Uiloolea
Siodauf
Rabio
Deji
Kai
Vi
